In [58]:
import pandas as pd


In [ ]:
reviews = pd.read_csv("reviews.csv", delimiter=",", encoding="latin1", nrows=10000)
recipes = pd.read_csv("recipes.csv", delimiter=",", encoding="latin1", nrows=10000)

In [60]:
reviews

,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z
...,...,...,...,...,...,...,...,...
9995,13087,23810,39282,kam23,0,"We used to go to a place in Litchfield, Illino...",2002-04-29T19:27:52Z,2002-04-29T19:27:52Z
9996,13088,13707,35526,riffraff,5,Very good. I did a test run on these this wee...,2002-04-29T19:27:53Z,2002-04-29T19:27:53Z
9997,13089,11489,35526,riffraff,5,This recipe was so good it even got a good rat...,2002-04-29T19:27:53Z,2002-04-29T19:27:53Z
9998,13090,23918,39285,Kimberly M,5,Excellent salad. And SO SIMPLE to make!! I m...,2002-04-29T19:27:54Z,2002-04-29T19:27:54Z


Création de la matrice Utilisateur-Recette

In [61]:
def create_user_item(dataframe):
    user_item = dataframe.pivot(index='AuthorId', columns='RecipeId', values='Rating')
    return user_item


In [ ]:
user_item_matrix = create_user_item(reviews)
user_item_matrix = user_item_matrix.fillna(0)
user_item_matrix.sum()

RecipeId
44        28.0
46         5.0
48         0.0
62         9.0
67         5.0
          ... 
66319     10.0
70137     28.0
73263      4.0
78151      5.0
109055     4.0
Length: 5514, dtype: float64

Calcul de la similarité entre utilisateurs

In [ ]:
user_item_matrix

RecipeId,44,46,48,62,67,72,83,89,100,102,...,53531,53607,54484,64306,64327,66319,70137,73263,78151,109055
AuthorId,,,,,,,,,,,,,,,,,,,,,
1533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1566,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1773,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1891,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39273,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcul de la similarité de cosinus entre les utilisateurs
user_similarity = cosine_similarity(user_item_matrix.fillna(0))
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)


SyntaxError: unexpected EOF while parsing (264331914.py, line 2)

In [ ]:
user_id = reviews['AuthorId']
num_recommendations = reviews['AuthorId'].shape

# Sélection des utilisateurs similaires
similar_users = user_similarity_df[user_id].sort_values(ascending=False)

# Recettes non évaluées par l'utilisateur
user_interactions = user_item_matrix.loc[user_id].dropna()
recommendations = {}

for similar_user, similarity in similar_users.iteritems():
    if similar_user != user_id:
        similar_user_ratings = user_item_matrix.loc[similar_user]
        for recipe_id, rating in similar_user_ratings.dropna().items():
            if recipe_id not in user_interactions:
                recommendations[recipe_id] = recommendations.get(recipe_id, 0) + similarity * rating

# Trier les recommandations par score
recommended_recipes = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:num_recommendations]
print("Recommandations de recettes :", recommended_recipes)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

true_ratings = []
predicted_ratings = []

for user_id in user_item_matrix.index:
    recommendations = recommended_recipes(user_id, user_item_matrix, user_similarity_df, num_recommendations=5)
    user_ratings = user_item_matrix.loc[user_id]

    for recipe_id in recommendations:
        true_ratings.append(user_ratings.get(recipe_id, np.nan))
        predicted_ratings.append(recommendations[recipe_id])

rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))
mae = mean_absolute_error(true_ratings, predicted_ratings)

print("RMSE :", rmse)
print("MAE :", mae)
